In [90]:
from pprint import pprint
import requests

TOKEN_API_OPENWEATHER = '16566c3a3219a0c615ec12412a4acaae'
city_ID = 'LONDON'
country_code = 'UK'
data = requests.get(f'http://history.openweathermap.org/data/2.5/history/city?q={city_ID},{country_code}&type=hour&appid={TOKEN_API_OPENWEATHER}').json()
# это перестало работать :(
# не дает взять данные за временной промежуток - хочет денег
# но доставать данные по api я научился


import os
os.environ['KAGGLE_USERNAME'] = "nkoluzanov"
os.environ['KAGGLE_KEY'] = "3985ee1aec78f857e9cf60d34c1af2c3"
!kaggle datasets download -d nguyenhoc/plane-crash
# unzip
import zipfile
path_to_extract = 'data/plane_crash_dataset/'
with zipfile.ZipFile('plane-crash.zip', 'r') as zip_ref:
    zip_ref.extractall(path_to_extract)

plane-crash.zip: Skipping, found more recently modified local copy (use --force to force download)


In [91]:
import pandas as pd
import numpy as np

In [92]:
data = pd.read_csv(os.path.join('data/plane_crash_dataset','planecrashinfo_20181121001952.csv'))

In [93]:
last_crashes = data.tail(1000)
last_crashes

,date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
4783,"July 30, 1998",15:58,"Near Quiberon, France",Proteus Air / Private,706,Lyon - Lorient,Beechcraft 1900D / Cessna 177,F-GSJM/F-GAJE,UE-238,15 (passengers:12 crew:3),15 (passengers:12 crew:3),0,Midair collision. The Beechcraft was on a flig...
4784,"July 30, 1998",11:05,"Kochi, India",Alliance Airlines,503,Agathi - Kochi - Thiruvananthapuram,Dornier 228-201,VT-EJW,8075/2017,6 (passengers:3 crew:3),6 (passengers:3 crew:3),2,"Shortly after taking off the plane pitched up,..."
4785,"August 04, 1998",17:58,"Kincolith, British Columbia, Canada",Harbour Air Seaplanes Ltd.,709,Prince Rupert - Kincolith,de Havilland Dash-2 Beaver,C-FOCJ,0039,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0,The pilot made three low passes over the water...
4786,"August 11, 1998",?,"Saurimo, Angola",Alada,?,?,Antonov An-12BP,D2-FAZ,?,13 (passengers:6 crew:7),1 (passengers:0 crew:1),0,Blew a tire while landing causing the plane to...
4787,"August 19, 1998",16:40,"Foveaux Strait, New Zealand",Vincent Aviation,?,Stewart Island - Invercargill,Cessna 402C,ZK-VAC,402C0512,10 (passengers:9 crew:1),5 (passengers:4 crew:1),0,"While en route, the aircraft experienced doubl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5778,"July 10, 2018",0744,"Pretoria, South Africa",Rovos Air,?,Test Flight,Convair CV-340,ZS-BRV,215,19 (passengers:16 crew:3),1 (passengers:0 crew:1),1,"Shortly after takeoff from runway 29, the left..."
5779,"August 04, 2018",1655,"Flims, Switzerland",Ju Air,?,Locorno - Dubendorf,Junkers JU-52,HB-HOT,6595,20 (passengers:17 crew:3),20 (passengers:17 crew:3),1,The vintage aircraft crashed onto Piz Segnas m...
5780,"September 28, 2018",1010,"Chuuk, Micronesia",Air Niugini,?,Pohnpei - Chuuk,Boeing 737-8BK,P2-PXE,33024/1688,47 (passengers:35 crew:12),1 (passengers:1 crew:0),0,The aircraft was approaching for a landing at ...
5781,"October 29, 2018",0631,"Off Jakarta, Indonesia",Lion Air,610,Jakarta - Pangkal Pinang,Boeing 737-MAX 8,PK-LQP,43000/7058,189 (passengers:181 crew:8),189 (passengers:181 crew:8),0,"The airliner crashed into the Jakarta Sea, 13 ..."


In [94]:
last_crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 4783 to 5782
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1000 non-null   object
 1   time          1000 non-null   object
 2   location      1000 non-null   object
 3   operator      1000 non-null   object
 4   flight_no     1000 non-null   object
 5   route         1000 non-null   object
 6   ac_type       1000 non-null   object
 7   registration  1000 non-null   object
 8   cn_ln         1000 non-null   object
 9   aboard        1000 non-null   object
 10  fatalities    1000 non-null   object
 11  ground        1000 non-null   object
 12  summary       1000 non-null   object
dtypes: object(13)
memory usage: 101.7+ KB


*standartize time date*

In [95]:
from datetime import datetime

datetime_object = datetime.strptime('July 30, 1998', '%B %d, %Y')
with np.nditer([last_crashes['date'], last_crashes['time']], flags=["refs_ok"], op_flags=['readwrite']) as it:
   for x,y in it:
       y = str(y).replace('?', '00:00').replace('c','')
       try:
            x[...] = datetime.strptime(str(x)+ ' ' + str(y), '%B %d, %Y %H:%M')
       except ValueError:
            x[...] = datetime.strptime(str(x) + ' 00:00', '%B %d, %Y %H:%M')

In [96]:

last_crashes.head()

,date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
4783,1998-07-30 15:58:00,15:58,"Near Quiberon, France",Proteus Air / Private,706,Lyon - Lorient,Beechcraft 1900D / Cessna 177,F-GSJM/F-GAJE,UE-238,15 (passengers:12 crew:3),15 (passengers:12 crew:3),0,Midair collision. The Beechcraft was on a flig...
4784,1998-07-30 11:05:00,11:05,"Kochi, India",Alliance Airlines,503,Agathi - Kochi - Thiruvananthapuram,Dornier 228-201,VT-EJW,8075/2017,6 (passengers:3 crew:3),6 (passengers:3 crew:3),2,"Shortly after taking off the plane pitched up,..."
4785,1998-08-04 17:58:00,17:58,"Kincolith, British Columbia, Canada",Harbour Air Seaplanes Ltd.,709,Prince Rupert - Kincolith,de Havilland Dash-2 Beaver,C-FOCJ,0039,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0,The pilot made three low passes over the water...
4786,1998-08-11 00:00:00,?,"Saurimo, Angola",Alada,?,?,Antonov An-12BP,D2-FAZ,?,13 (passengers:6 crew:7),1 (passengers:0 crew:1),0,Blew a tire while landing causing the plane to...
4787,1998-08-19 16:40:00,16:40,"Foveaux Strait, New Zealand",Vincent Aviation,?,Stewart Island - Invercargill,Cessna 402C,ZK-VAC,402C0512,10 (passengers:9 crew:1),5 (passengers:4 crew:1),0,"While en route, the aircraft experienced doubl..."


In [97]:
last_crashes.index = pd.to_datetime(last_crashes['date'])

In [98]:
# last_crashes = last_crashes.drop(columns='date')

In [99]:
last_crashes.head()

,date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
date,,,,,,,,,,,,,
1998-07-30 15:58:00,1998-07-30 15:58:00,15:58,"Near Quiberon, France",Proteus Air / Private,706,Lyon - Lorient,Beechcraft 1900D / Cessna 177,F-GSJM/F-GAJE,UE-238,15 (passengers:12 crew:3),15 (passengers:12 crew:3),0,Midair collision. The Beechcraft was on a flig...
1998-07-30 11:05:00,1998-07-30 11:05:00,11:05,"Kochi, India",Alliance Airlines,503,Agathi - Kochi - Thiruvananthapuram,Dornier 228-201,VT-EJW,8075/2017,6 (passengers:3 crew:3),6 (passengers:3 crew:3),2,"Shortly after taking off the plane pitched up,..."
1998-08-04 17:58:00,1998-08-04 17:58:00,17:58,"Kincolith, British Columbia, Canada",Harbour Air Seaplanes Ltd.,709,Prince Rupert - Kincolith,de Havilland Dash-2 Beaver,C-FOCJ,0039,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0,The pilot made three low passes over the water...
1998-08-11 00:00:00,1998-08-11 00:00:00,?,"Saurimo, Angola",Alada,?,?,Antonov An-12BP,D2-FAZ,?,13 (passengers:6 crew:7),1 (passengers:0 crew:1),0,Blew a tire while landing causing the plane to...
1998-08-19 16:40:00,1998-08-19 16:40:00,16:40,"Foveaux Strait, New Zealand",Vincent Aviation,?,Stewart Island - Invercargill,Cessna 402C,ZK-VAC,402C0512,10 (passengers:9 crew:1),5 (passengers:4 crew:1),0,"While en route, the aircraft experienced doubl..."


In [100]:

last_crashes

,date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
date,,,,,,,,,,,,,
1998-07-30 15:58:00,1998-07-30 15:58:00,15:58,"Near Quiberon, France",Proteus Air / Private,706,Lyon - Lorient,Beechcraft 1900D / Cessna 177,F-GSJM/F-GAJE,UE-238,15 (passengers:12 crew:3),15 (passengers:12 crew:3),0,Midair collision. The Beechcraft was on a flig...
1998-07-30 11:05:00,1998-07-30 11:05:00,11:05,"Kochi, India",Alliance Airlines,503,Agathi - Kochi - Thiruvananthapuram,Dornier 228-201,VT-EJW,8075/2017,6 (passengers:3 crew:3),6 (passengers:3 crew:3),2,"Shortly after taking off the plane pitched up,..."
1998-08-04 17:58:00,1998-08-04 17:58:00,17:58,"Kincolith, British Columbia, Canada",Harbour Air Seaplanes Ltd.,709,Prince Rupert - Kincolith,de Havilland Dash-2 Beaver,C-FOCJ,0039,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0,The pilot made three low passes over the water...
1998-08-11 00:00:00,1998-08-11 00:00:00,?,"Saurimo, Angola",Alada,?,?,Antonov An-12BP,D2-FAZ,?,13 (passengers:6 crew:7),1 (passengers:0 crew:1),0,Blew a tire while landing causing the plane to...
1998-08-19 16:40:00,1998-08-19 16:40:00,16:40,"Foveaux Strait, New Zealand",Vincent Aviation,?,Stewart Island - Invercargill,Cessna 402C,ZK-VAC,402C0512,10 (passengers:9 crew:1),5 (passengers:4 crew:1),0,"While en route, the aircraft experienced doubl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-10 00:00:00,2018-07-10 00:00:00,0744,"Pretoria, South Africa",Rovos Air,?,Test Flight,Convair CV-340,ZS-BRV,215,19 (passengers:16 crew:3),1 (passengers:0 crew:1),1,"Shortly after takeoff from runway 29, the left..."
2018-08-04 00:00:00,2018-08-04 00:00:00,1655,"Flims, Switzerland",Ju Air,?,Locorno - Dubendorf,Junkers JU-52,HB-HOT,6595,20 (passengers:17 crew:3),20 (passengers:17 crew:3),1,The vintage aircraft crashed onto Piz Segnas m...
2018-09-28 00:00:00,2018-09-28 00:00:00,1010,"Chuuk, Micronesia",Air Niugini,?,Pohnpei - Chuuk,Boeing 737-8BK,P2-PXE,33024/1688,47 (passengers:35 crew:12),1 (passengers:1 crew:0),0,The aircraft was approaching for a landing at ...


In [101]:
last_crashes.index = last_crashes.index.to_period('D')

In [102]:
last_crashes.head()

,date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
date,,,,,,,,,,,,,
1998-07-30,1998-07-30 15:58:00,15:58,"Near Quiberon, France",Proteus Air / Private,706,Lyon - Lorient,Beechcraft 1900D / Cessna 177,F-GSJM/F-GAJE,UE-238,15 (passengers:12 crew:3),15 (passengers:12 crew:3),0,Midair collision. The Beechcraft was on a flig...
1998-07-30,1998-07-30 11:05:00,11:05,"Kochi, India",Alliance Airlines,503,Agathi - Kochi - Thiruvananthapuram,Dornier 228-201,VT-EJW,8075/2017,6 (passengers:3 crew:3),6 (passengers:3 crew:3),2,"Shortly after taking off the plane pitched up,..."
1998-08-04,1998-08-04 17:58:00,17:58,"Kincolith, British Columbia, Canada",Harbour Air Seaplanes Ltd.,709,Prince Rupert - Kincolith,de Havilland Dash-2 Beaver,C-FOCJ,0039,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0,The pilot made three low passes over the water...
1998-08-11,1998-08-11 00:00:00,?,"Saurimo, Angola",Alada,?,?,Antonov An-12BP,D2-FAZ,?,13 (passengers:6 crew:7),1 (passengers:0 crew:1),0,Blew a tire while landing causing the plane to...
1998-08-19,1998-08-19 16:40:00,16:40,"Foveaux Strait, New Zealand",Vincent Aviation,?,Stewart Island - Invercargill,Cessna 402C,ZK-VAC,402C0512,10 (passengers:9 crew:1),5 (passengers:4 crew:1),0,"While en route, the aircraft experienced doubl..."


In [103]:
# last_crashes = last_crashes.resample('M',convention='end').mean()

In [106]:
from sktime.utils.plotting import plot_series

plot_series(last_crashes['date'])

TypeError: No valid mtype could be identified for object of type <class 'pandas.core.series.Series'>. Errors returned are as follows, in format [mtype]: [error message] 
pd.DataFrame: obj must be a pandas.DataFrame, found <class 'pandas.core.series.Series'>
pd.Series: obj should not be of 'object' dtype
np.ndarray: obj must be a numpy.ndarray, found <class 'pandas.core.series.Series'>
